In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr,chi2_contingency
import matplotlib.pyplot as plt
import tools

In [2]:
df=pd.read_csv('train.csv')

In [3]:
X=df.treatment
Y=df.outcome
S=set()
col_set={i for i in df.columns if 'V' in i}

In [4]:
for i in df.columns:
    s=df[i]
    if len(s.unique())<10:
        #处理离散变量
        s=np.array(pd.get_dummies(s)).argmax(-1)
        df[i]=s
    else:
        #处理连续变量
        df[i]=df[i].fillna(df[i].mean())

In [5]:
#根据相关性选取不为A的变量
for col in col_set-S:
    s=df[col]
    c,p=spearmanr(s,Y)
    if p<0.00001:
        print(col,'Y',c,p)
        S.add(col)
for col in col_set-S:
    s=df[col]
    c,p=spearmanr(s,X)
    if p<0.00001:
        print(col,'X',c,p)
        S.add(col)

/home/oybb/anaconda3/envs/torch/lib/python3.7/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


V_36 Y 0.07045666946412761 4.661666090848561e-41
V_8 Y 0.13224317988172335 7.370815294795673e-141
V_28 Y -0.034471038805503375 5.4097952629401624e-11
V_31 Y 0.22687592431776285 0.0
V_29 Y -0.05924019675958815 1.6688663786725704e-29
V_10 Y -0.0908128288531666 3.910803326695209e-67
V_6 Y -0.09979195694517497 9.546884520387867e-81
V_14 Y -0.20517894758604227 0.0
V_37 Y -0.25132970296789325 0.0
V_15 Y 0.37477123768005083 0.0
V_7 Y -0.281769726167632 0.0
V_21 Y -0.04147341172671237 2.957893009953722e-15
V_5 Y -0.09876740782682815 3.969645277413074e-79
V_1 Y 0.11629697297448234 3.596237913409036e-109
V_2 Y -0.0702837617561886 7.273878052260199e-41
V_19 Y 0.167631799705178 2.7026127433060447e-226
V_16 Y -0.11270689774546816 1.3043175767652795e-102
V_33 Y 0.1429899115716578 1.3880019097458658e-164
V_25 Y -0.26553132849998906 0.0
V_23 Y 0.04738125029039025 1.912804698576465e-19
V_9 Y -0.2653332938763218 0.0
V_30 Y -0.9907080947821001 0.0
V_26 Y -0.053740953813899275 1.4513144130582155e-24
V_32 

In [6]:
#根据相关性进一步选取不为A的变量
for i in col_set-S:
    a=df[i]
    lis=[spearmanr(a,df[col])[1] for col in S]
    if min(lis)<1e-5:
        print(i)
        S.add(i)

V_0
V_34
V_4


In [7]:
#这几个没有明显证据证明不属于A
col_set-S

{'V_17', 'V_20', 'V_27', 'V_38'}

In [8]:
l=list()
for i in S:
    if len(df[i].unique())<10:
        l.append(i)

In [9]:
#看看能不能恰好运气好找出d分离，恰好测出来了，V_16大概率属于W
L=list()
treatment=2
for col in S:
    a=df[col][df.treatment==treatment]
    y=Y[df.treatment==treatment]
    if spearmanr(a,y)[1]<0.001:
        for w_col in l:
            w=df[w_col][df.treatment==treatment]
            cond=False
            for value in w.unique():
                cond=cond or spearmanr(a[w==value],y[w==value])[1]<0.05
            if not cond:
                L.append((w_col,col))
                print(w_col,col)

V_8 V_8
V_10 V_10
V_14 V_14
V_15 V_15
V_16 V_15
V_16 V_16
V_12 V_12
V_16 V_9
V_26 V_26
V_16 V_35
V_39 V_39
V_15 V_22
V_16 V_22
V_5 V_5
